In [ ]:
!pip install dwavebinarycsp

In [ ]:
!pip install dwave-system

In [ ]:
!pip install dwave-inspector

In [ ]:
import math

import dwavebinarycsp
import dwave.inspector
from dwave.system import EmbeddingComposite, DWaveSampler

In [ ]:
# Copyright 2020 D-Wave Systems Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Handy functions that are not necessary for the conceptual understanding of
# this code example

from collections import defaultdict

import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.cluster import KMeans
import requests
import math


def find_nearest_cluster_for_a_point(cluster1, cluster2, color1, color2, point):
    distance1 = 0
    distance2 = 0
    print("@@@@Cluster1=" + str(cluster1) + '&&&&&&&&& point=' + str(point))
    for item in cluster1:
        diff_x = item[0]-point[0]
        diff_y = item[1]-point[1]
        diff_z = item[2]-point[2]
        d = math.sqrt(diff_x**2 + diff_y**2 + diff_z**2)
        distance1 = distance1 +d

    for item in cluster2:
        diff_x = item[0]-point[0]
        diff_y = item[1]-point[1]
        diff_z = item[2]-point[2]
        d = math.sqrt(diff_x**2 + diff_y**2 + diff_z**2)
        distance2 = distance2 +d
    if distance1 <= distance2:
        color = color1
    else: color = color2

    return color


def get_groupings(sample):
    """Grab selected items and group them by color"""
    colored_points = defaultdict(list)
    print("@@@@@@SAMPLE=" + str(sample))
    array_all_coordinates = []
    array_all_colors = []

    '''if len(array_all_coordinates) == 0:
            array_all_coordinates.append(coord)
    else:
        for item in array_all_coordinates:
                print("@@@@@array not duplicate="+ str(array_all_coordinates))
                if item == coord:
                    print("duplicate")
                else:
                    array_all_coordinates.append(coord)'''


    for label, bool_val in sample.items():

        # Skip over items that were not selected
        if not bool_val:
            continue
        #print("bool_val" + str(bool_val))
        # Parse selected items
        # Note: label look like "<x_coord>,<y_coord>_<color>"
        coord, color = label.split("_")
        #print("$$$$$ label = " + str(label))
        #array_all_coordinates.append(coord)
        if len(array_all_coordinates) == 0:
            array_all_coordinates.append(coord)
            array_all_colors.append(color)

        else:
            k=0
            for j in range(0,len(array_all_coordinates)):
                if array_all_coordinates[j]==coord:
                    k=1
                    number_of_red = 0
                    number_of_blue = 0
                    number_of_green = 0
                    coord_reds = []
                    coord_blue = []
                    coord_green = []
                    for mm in range (0, len(array_all_colors)):
                        if array_all_colors[mm]=='red':
                            number_of_red = number_of_red+1
                            coord_reds.append(tuple(map(float,array_all_coordinates[mm].split(","))))
                        elif array_all_colors[mm]=='green':
                            number_of_green = number_of_green + 1
                            coord_green.append(tuple(map(float,array_all_coordinates[mm].split(","))))
                        elif array_all_colors[mm]=='blue':
                            number_of_blue= number_of_blue + 1
                            coord_blue.append(tuple(map(float,array_all_coordinates[mm].split(","))))
                    current_exist_color = array_all_colors[j]
                    print('current_exist_color' + current_exist_color + ', and color=' + color)

                    if current_exist_color==color:
                        print('do nothing')
                    elif (current_exist_color =='red' and color=='green' and number_of_green < number_of_red):
                        final_color = find_nearest_cluster_for_a_point(coord_reds, coord_green, 'red', 'green', tuple(map(float,coord.split(","))))
                        print('in the 1th condition')
                        array_all_colors[j] = final_color
                    elif (current_exist_color =='red' and color=='blue' and number_of_blue < number_of_red):
                        final_color= find_nearest_cluster_for_a_point(coord_reds, coord_blue, 'red', 'blue',tuple(map(float,coord.split(","))))
                        print('in the 2th condition')
                        array_all_colors[j] = final_color
                    elif (current_exist_color =='blue' and color=='green' and number_of_green < number_of_blue):
                        final_color= find_nearest_cluster_for_a_point(coord_blue, coord_green, 'blue', 'green', tuple(map(float,coord.split(","))))
                        print('in the 3th condition')
                        array_all_colors[j] = final_color
                    elif (current_exist_color =='blue' and color=='red' and number_of_red < number_of_blue):
                        final_color= find_nearest_cluster_for_a_point(coord_blue, coord_reds, 'blue', 'red',tuple(map(float,coord.split(","))))
                        print('in the 4th condition')
                        array_all_colors[j] = final_color
                    elif (current_exist_color =='green' and color=='blue' and number_of_blue < number_of_green):
                        final_color= find_nearest_cluster_for_a_point(coord_green, coord_blue, 'green', 'blue', tuple(map(float,coord.split(","))))
                        print('in the 5th condition')
                        array_all_colors[j] = final_color
                    elif (current_exist_color =='green' and color=='red' and number_of_red < number_of_green):
                        final_color= find_nearest_cluster_for_a_point(coord_green, coord_reds, 'green', 'red', tuple(map(float,coord.split(","))))
                        print('in the 6th condition')
                        array_all_colors[j] = final_color

                    print('duplicate for j=' + str(j))
            if k==0:
                array_all_coordinates.append(coord)
                array_all_colors.append(color)


        #coord_tuple = tuple(map(float, coord.split(",")))
        #colored_points[color].append(coord_tuple)
    #print("$$$$$$$$$$$array not duplicate="+ str(array_all_coordinates))
    '''j=0
    array_no_duplicate=[]
    for i in range(0, len(array_all_coordinates)):
        k=0
        for j in range(i+1,len(array_all_coordinates)-1):
            if array_all_coordinates[i]==array_all_coordinates[j]:
                k=1
                print('duplicate for i=' + str(i) + ', j=' + str(j))
        if k==0:
            array_no_duplicate.append(array_all_coordinates[i])

    print("@@@@@@@@@array_no_duplicate=" + str(array_no_duplicate))'''
    my_colored_points = defaultdict(list)
    for k in range(0, len(array_all_coordinates)):
        coord_tuple = tuple(map(float, array_all_coordinates[k].split(",")))
        my_colored_points[array_all_colors[k]].append(coord_tuple)
    return dict(my_colored_points)



def visualize_groupings(groupings_dict, filename):
    """
    Args:
        groupings_dict: key is a color, value is a list of x-y coordinate tuples.
          For example, {'r': [(0,1), (2,3)], 'b': [(8,3)]}
        filename: name of the file to save plot in
    """

    fig = plt.figure()
    ax = plt.axes(projection = '3d')
    for color, points in groupings_dict.items():
        print("color=" + str(color))
        print("points=" + str(points))
        # Ignore items that do not contain any coordinates
        if not points:
            continue

        for item in points:
            print("item x=" + str(item[0]))
            if color == 'blue':
                #print('color2=' + str(color))
                ax.scatter3D(item[0],item[1],item[4], color='blue')
            elif color == 'red':
               # print('color2=' + str(color))
                ax.scatter3D(item[0],item[1],item[4], color='red')
            elif color == 'green':
               # print('color2=' + str(color))
                ax.scatter3D(item[0],item[1],item[4], color='green')

        # Give labels
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        # Save figure


    plt.savefig(filename)


def visualize_scatterplot(x_y_tuples_list, filename):
    """Plotting out a list of x-y tuples

    Args:
        x_y_tuples_list: A list of x-y coordinate values. e.g. [(1,4), (3, 2)]
    """
    plt.axes(projection='3d')
    plt.plot(*zip(*x_y_tuples_list), "o")
    plt.savefig(filename)


def initialize_three_points(scattered_points):
    size_array = len(scattered_points)
    # choose three indexes between 0 to size_array-1
    a, b, c = random.sample(range(size_array), 3)

    center_a = scattered_points[a]
    center_b = scattered_points[b]
    center_c = scattered_points[c]


    return center_a, center_b, center_c

def calculate_centroid_of_cluster(c):
    array_points = []
    for item in c:
        x= item.x
        y= item.y
        z= item.z
    return centroid



def visualize_elbow(scattered_points):


    #data = list(zip(x, y))
    inertias = []

    for i in range(1,min(len(scattered_points), 15)):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(scattered_points)
        inertias.append(kmeans.inertia_)

    plt.plot(range(1,min(len(scattered_points), 15)), inertias, marker='o')
    plt.title('Elbow method')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.savefig("Elbow")

def run_kmeans_on_classical_computer(scattered_points):
    kmeans = KMeans(n_clusters=3)
    res = kmeans.fit(scattered_points)
    print("result classical=" + str(res))
    print("result classical labels=" + str(kmeans.labels_))

    ax = plt.axes(projection = '3d')
    #ax.scatter3D(x, y, z, c=kmeans.labels_)
    #plt.plot(*zip(*scattered_points), c=kmeans.labels_)
    kk = 0
    for item in scattered_points:
            if kmeans.labels_[kk] == 0:
                ax.scatter3D(item[0],item[1],item[3], color='blue')
            elif kmeans.labels_[kk] == 1:
                ax.scatter3D(item[0],item[1],item[3], color='red')
            elif kmeans.labels_[kk] == 2:
                ax.scatter3D(item[0],item[1],item[3], color='green')
            kk =kk+1
    #ax.scatter3D(*zip(*scattered_points), c=kmeans.labels_)
    plt.savefig("result_classical")
    with open("/content/result_classical.txt", "a+") as file_object:

        file_object.write("\n")
        # Append text at the end of file
        file_object.write(str(kmeans.labels_))


def get_bike_stations():

    response = requests.get("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information")
    response2 = requests.get("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status")
    data = response.json()
    data2 = response2.json()

    array_stations = data['data']['stations']
    array_stations_status = data2['data']['stations']
    array_population_near_each_station = [65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180,65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180, 65913,31340,7607,30526,17945,25797,25797,17945,25797,31340, 65913,65913,25797,20506, 20506,25797, 65913,65913,
    65913,17945,17945,25797,25797,7607,31340,18615,25797,17945,31340,25797,25797,18615,17945,65913,20923,31180,65913,16556,25797,17945,30526,25797,
    18615,65913,31340,31340,31180,25797,65913,65913,31180]

    tuple_lat_lon_capacity = []


    ## num_docks_available = capacity-[num_bikes_available+num_bikes_disabled+num_docks_disabled]
    #for i in range(0, len(array_stations)):
    print("len =" + str(len(array_stations)))
    number_of_stations = len(array_stations)
    for i in range(1, 51):
        i = i*2
        print('i=' + str(i))
        tuple_lat_lon_capacity.append((array_stations[number_of_stations-i]['lat'], array_stations[number_of_stations-i]['lon'],array_stations[number_of_stations-i]['capacity'],array_stations_status[number_of_stations-i]['num_bikes_available'],
        array_population_near_each_station[number_of_stations-i]))

    print("tuple_lat_lon_capacity=" + str(tuple_lat_lon_capacity))

    return tuple_lat_lon_capacity

In [ ]:
class Coordinate:
    def __init__(self, x, y, z, nba, pns):
        self.x = x
        self.y = y
        self.z = z
        self.nba = nba
        self.pns = pns

        # coordinate labels for groups red, green, and blue
        label = "{0},{1},{2},{3},{4}_".format(x, y, z, nba, pns)
        self.r = label + "red"
        self.g = label + "green"
        self.b = label + "blue"


def get_distance(coordinate_0, coordinate_1):
    diff_x = coordinate_0.x - coordinate_1.x
    diff_y = coordinate_0.y - coordinate_1.y
    diff_z = coordinate_0.z - coordinate_1.z
    diff_nba = coordinate_0.nba - coordinate_1.nba
    diff_pns = coordinate_0.pns - coordinate_1.pns

    return math.sqrt(diff_x**2 + diff_y**2 + diff_z**2 + diff_nba**2 + diff_pns**2)


def get_max_distance(coordinates):
    max_distance = 0
    for i, coord0 in enumerate(coordinates[:-1]):
        for coord1 in coordinates[i+1:]:
            distance = get_distance(coord0, coord1)
            max_distance = max(max_distance, distance)
    return max_distance


In [ ]:
!dwave config create --full

Updating existing configuration file: /root/.config/dwave/dwave.conf
Confirm configuration file path [/root/.config/dwave/dwave.conf]: 
Available profiles: defaults, create
Profile (select existing or create new) [defaults]: 
Updating existing profile: defaults
Solver API region (na-west-1, eu-central-1) [skip]: 
Solver API endpoint URL (overwrites 'region') [region]: https://na-west-1.cloud.dwavesys.com/sapi/v2/
Solver API token [DEV-49c85af23caf299303d14889afaa960c6204a821]: 
Client class (base, qpu, sw, hybrid) [skip]: 
Solver [skip]: 
Configuration saved.


In [ ]:
def find_nearest_cluster_to_points(coordinates, centroid_a, centroid_b, centroid_c):
    """ For each point we need to check the distance to the three centroids and find the closest one.
    """
    cluster_1 = []
    cluster_2 = []
    cluster_3 = []
    #centroid_a= Coordinate(centroid_a[0], centroid_a[1], centroid_a[2])
    #centroid_b= Coordinate(centroid_b[0], centroid_b[1], centroid_b[2])
    #centroid_c= Coordinate(centroid_c[0], centroid_c[1], centroid_c[2])

    #coordinates = [Coordinate(x, y, z) for x, y, z in scattered_points]
    closest_distance = 1000000

    for i, coord0 in enumerate(coordinates):
        distance_1 = get_distance(coord0, centroid_a)
        distance_2 = get_distance(coord0, centroid_b)
        distance_3 = get_distance(coord0, centroid_c)
        closest_distance = min(distance_1,distance_2,distance_3)
        print("closest distance=" + str(closest_distance))
        if distance_1 == closest_distance:
            cluster_1.append(coord0)
        elif distance_2 == closest_distance:
            cluster_2.append(coord0)
        elif distance_3 == closest_distance:
            cluster_3.append(coord0)

    return cluster_1, cluster_2, cluster_3



def cluster_points(scattered_points, filename, problem_inspector):
    """Perform clustering analysis on given points

    Args:
        scattered_points (list of tuples):
            Points to be clustered
        filename (str):
            Output file for graphic
        problem_inspector (bool):
            Whether to show problem inspector
    """
    # Set up problem
    # Note: max_distance gets used in division later on. Hence, the max(.., 1)
    #   is used to prevent a division by zero
    # x is lat, y is lon, z is capacity, nba is number of bikes available, pns is population near each station
    ####### I want to calculate embedding time
    start_time_emb = time.time()
    #scattered_points = get_bike_stations()
    print("scattered_points=" + str(scattered_points))
    coordinates = [Coordinate(x, y, z, nba, pns) for x, y, z, nba, pns in scattered_points]
    print("coordinates=" + str(coordinates))

    max_distance = max(get_max_distance(coordinates), 1)
    print("MAx distance=" + str(max_distance))

    # Build constraints
    csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)

    # Apply constraint: coordinate can only be in one colour group
    choose_one_group = {(0, 0, 1), (0, 1, 0), (1, 0, 0)}
    for coord in coordinates:
        #print('coord.r=' + str(coord.r))
        #print('coord.g=' + str(coord.g))
        csp.add_constraint(choose_one_group, (coord.r, coord.g, coord.b))

    # Build initial BQM
    bqm = dwavebinarycsp.stitch(csp)

    # Edit BQM to bias for close together points to share the same color
    for i, coord0 in enumerate(coordinates[:-1]):
        for coord1 in coordinates[i+1:]:
            # Set up weight
            d = get_distance(coord0, coord1) / max_distance  # rescale distance
            weight = -math.cos(d*math.pi)
            #print('weight='+str(weight))

            # Apply weights to BQM
            bqm.add_interaction(coord0.r, coord1.r, weight)
            bqm.add_interaction(coord0.g, coord1.g, weight)
            bqm.add_interaction(coord0.b, coord1.b, weight)

    # Edit BQM to bias for far away points to have different colors
    for i, coord0 in enumerate(coordinates[:-1]):
        for coord1 in coordinates[i+1:]:
            # Set up weight
            # Note: rescaled and applied square root so that far off distances
            #   are all weighted approximately the same
            d = math.sqrt(get_distance(coord0, coord1) / max_distance)
            weight = -math.tanh(d) * 0.1
            #print("weight2=" + str(weight))

            # Apply weights to BQM
            bqm.add_interaction(coord0.r, coord1.b, weight)
            bqm.add_interaction(coord0.r, coord1.g, weight)
            bqm.add_interaction(coord0.b, coord1.r, weight)
            bqm.add_interaction(coord0.b, coord1.g, weight)
            bqm.add_interaction(coord0.g, coord1.r, weight)
            bqm.add_interaction(coord0.g, coord1.b, weight)
            #print("step1")
    # Submit problem to D-Wave sampler
    #print("#")
    sampler = EmbeddingComposite(DWaveSampler())
    print("bqm= " + str(bqm))
    sampleset = sampler.sample(bqm,
                               chain_strength=4,
                               num_reads=1000  ,
                               label='Bike Sharing Clustering')
    print("####Embedding time=", time.time() - start_time_emb)
    #print("step2")
    best_sample = sampleset.first.sample
    #print("step3")

    # Visualize graph problem
    if problem_inspector:
        dwave.inspector.show(bqm, sampleset)

    # Visualize solution
    groupings = get_groupings(best_sample)
    print("best_sample=" +str(best_sample))
    print("groupings=" + str(groupings))
    visualize_groupings(groupings, filename)

    with open("/content/result_QC.txt", "a+") as file_object:

        file_object.write("\n")
        # Append text at the end of file
        file_object.write(str(groupings))


In [ ]:
scattered_stations = get_bike_stations()

len =793
i=2
i=4
i=6
i=8
i=10
i=12
i=14
i=16
i=18
i=20
i=22
i=24
i=26
i=28
i=30
i=32
i=34
i=36
i=38
i=40
i=42
i=44
i=46
i=48
i=50
i=52
i=54
i=56
i=58
i=60
i=62
i=64
i=66
i=68
i=70
i=72
i=74
i=76
i=78
i=80
i=82
i=84
i=86
i=88
i=90
i=92
i=94
i=96
i=98
i=100
tuple_lat_lon_capacity=[(43.673391524931525, -79.47798367814572, 16, 7, 31340), (43.649354475273796, -79.39675713332855, 12, 3, 18615), (43.63906470236988, -79.41080996164456, 24, 12, 30526), (43.64286370000001, -79.40713060000002, 23, 6, 25797), (43.697417070558615, -79.50079830912324, 19, 2, 65913), (43.74445257991756, -79.56824985721664, 15, 4, 20923), (43.721532966873205, -79.5455834809678, 18, 25, 17945), (43.64781808540915, -79.38427999073791, 17, 9, 25797), (43.68233679999999, -79.4680594, 15, 3, 31340), (43.662452744495575, -79.3978384219513, 23, 32, 25797), (43.647968822765, -79.38316961352653, 19, 2, 31340), (43.70503461857136, -79.33547292037659, 23, 6, 25797), (43.650297574961044, -79.36648460341635, 14, 1, 17945), (43.663

In [ ]:
# I want to run it for 10 iterations and calculate the runtime
import time

for i in range(10):
    start_time = time.time()
    run_kmeans_on_classical_computer(scattered_stations)
    print("--- %s run time seconds ---" % (time.time() - start_time))


result classical=KMeans(n_clusters=3)
result classical labels=[1 2 1 1 0 2 2 1 1 1 1 1 2 0 0 2 1 0 1 1 2 2 0 0 1 1 0 1 2 2 1 0 2 1 2 2 2
 1 2 0 1 2 0 1 2 1 1 1 1 0]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


--- 0.9439511299133301 run time seconds ---
result classical=KMeans(n_clusters=3)
result classical labels=[2 0 2 2 1 0 0 2 2 2 2 2 0 1 1 0 2 1 2 2 0 0 1 1 2 2 1 2 0 0 2 1 0 2 0 0 0
 2 0 1 2 0 1 2 0 2 2 2 2 1]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


--- 1.257324457168579 run time seconds ---
result classical=KMeans(n_clusters=3)
result classical labels=[0 2 0 0 1 2 2 0 0 0 0 0 2 1 1 2 0 1 0 0 2 2 1 1 0 0 1 0 2 2 0 1 2 0 2 2 2
 0 2 1 0 2 1 0 2 0 0 0 0 1]


...

--- 5.078436613082886 run time seconds ---
result classical=KMeans(n_clusters=3)
result classical labels=[2 0 2 2 1 0 0 2 2 2 2 2 0 1 1 0 2 1 2 2 0 0 1 1 2 2 1 2 0 0 2 1 0 2 0 0 0
 2 0 1 2 0 1 2 0 2 2 2 2 1]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


--- 4.607003688812256 run time seconds ---
result classical=KMeans(n_clusters=3)
result classical labels=[0 2 0 0 1 2 2 0 0 0 0 0 2 1 1 2 0 1 0 0 2 2 1 1 0 0 1 0 2 2 0 1 2 0 2 2 2
 0 2 1 0 2 1 0 2 0 0 0 0 1]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


--- 5.312826156616211 run time seconds ---


In [ ]:

# step 1 randomly choose k examples as initial centroids
centroid_a, centroid_b, centroid_c = initialize_three_points(scattered_stations)
print("a=" + str(centroid_a) + ", b=" + str(centroid_b) + ", c=" + str(centroid_c))

    # step 2 assign each item to the closet centroid
centroid_aa= Coordinate(centroid_a[0], centroid_a[1], centroid_a[2], centroid_a[3], centroid_a[4])
centroid_bb= Coordinate(centroid_b[0], centroid_b[1], centroid_b[2], centroid_a[3], centroid_a[4])
centroid_cc= Coordinate(centroid_c[0], centroid_c[1], centroid_c[2], centroid_a[3], centroid_a[4])
coordinates = [Coordinate(x, y, z, f, g) for x, y, z, f, g in scattered_stations]
cluster_1, cluster_2, cluster_3 = find_nearest_cluster_to_points(coordinates, centroid_aa, centroid_bb, centroid_cc)
print("cluster1=" + str(cluster_1))
print("cluster2=" + str(cluster_2))
print("cluster3=" + str(cluster_3))


a=(43.645389479325615, -79.5276822170925, 15, 6, 17945), b=(43.74386580997666, -79.49181654358595, 15, 0, 31180), c=(43.60191488118503, -79.49941888654288, 15, 9, 65913)
closest distance=13395.000074776188
closest distance=670.0134422457139
closest distance=12581.00465023488
closest distance=7852.004076042876
closest distance=47968.00033357903
closest distance=2978.0006725725225
closest distance=19.23547217095116
closest distance=7852.0008287928795
closest distance=13395.000336108626
closest distance=7852.047122501458
closest distance=13395.001195059105
closest distance=7852.0040770462665
closest distance=5.100981514246782
closest distance=47968.00017731843
closest distance=47968.00000004179
closest distance=2561.0039076437947
closest distance=7852.001656598958
closest distance=47968.000375315525
closest distance=7852.0010840261675
closest distance=7852.002039013231
closest distance=4.123164258914321
closest distance=10338.038354031349
closest distance=47968.0003342434
closest distance

In [ ]:
pip install dwave-inspectorapp --extra-index=https://pypi.dwavesys.com/simple

Looking in indexes: https://pypi.org/simple, https://pypi.dwavesys.com/simple


In [ ]:
clustered_filename = "QC_clustered.png"

## I want to run it for 10 iterations
#for i in range(10):
cluster_points(scattered_stations, clustered_filename, True)


scattered_points=[(43.673391524931525, -79.47798367814572, 16, 7, 31340), (43.649354475273796, -79.39675713332855, 12, 3, 18615), (43.63906470236988, -79.41080996164456, 24, 12, 30526), (43.64286370000001, -79.40713060000002, 23, 6, 25797), (43.697417070558615, -79.50079830912324, 19, 2, 65913), (43.74445257991756, -79.56824985721664, 15, 4, 20923), (43.721532966873205, -79.5455834809678, 18, 25, 17945), (43.64781808540915, -79.38427999073791, 17, 9, 25797), (43.68233679999999, -79.4680594, 15, 3, 31340), (43.662452744495575, -79.3978384219513, 23, 32, 25797), (43.647968822765, -79.38316961352653, 19, 2, 31340), (43.70503461857136, -79.33547292037659, 23, 6, 25797), (43.650297574961044, -79.36648460341635, 14, 1, 17945), (43.66335070995886, -79.41312798316125, 11, 5, 65913), (43.675531973168354, -79.47199740066374, 15, 6, 65913), (43.656038841714505, -79.38970387738495, 11, 4, 20506), (43.65748153286412, -79.38921396018827, 16, 1, 25797), (43.69105005197055, -79.43281746494293, 15, 0, 